# C and Other Extensions

One of the great things about Python is that it is not the best tool for every job, and more
importantly, it knows that it isn't the best tool for every job. Because of this self-awareness,
Python was designed from the beginning to be extensible with code written in C. This
capability is provided by a module called Cython, which is available from http://cython.
org . In this chapter, you will look at some of the different ways you can include Cython within
your own Python programs in order to improve its performance or add extra functionality.
### 11-1. Compiling Python Code

You want to compile your Python code to C to get a speedup.
<br>
The Cython package provides a mechanism for mixing compiled C code with Python.
<br>
The initial setup applies to all of the following examples within this chapter. You need
to have a C compiler on your system. If you are running Linux or Mac OS, then you can
use gcc as the required compiler. To get the compiler on Mac OS, you need to install
the XCode package. As for Windows, there are more steps involved. There is an entire
appendix within the Cython documentation just for instructions on how to set up
Windows. As well, you will need Cython installed. You can either install it from source, or
you can install it using pip, as in Listing 11-1 .
#### Listing 11-1. Installing Cython with pip

In [1]:
pip install --user cython


The following command must be run outside of the IPython shell:

    $ pip install --user cython

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


Once everything is installed, you need to write the Python code that is to be compiled
into C. This code is saved in a file ending with .pyx , rather than .py . This Cython source
file is then compiled in one of several different ways. For larger projects, the most flexible
and robust way to handle the compilation is to write a setup.py file and use distutils .
This method is a bit too complex to introduce in such a short space. Happily, there are a
couple of other simpler methods to start using Cython with your code right away.
Listing 11-2 shows a sample . pyx file that just has a single function in it.
#### Listing 11-2. HelloWorld.pyx File

In [ ]:
def print_msg():
print("Hello World")

Cython includes a module named pyximport , which will compile . pyx files in the
background when you try to import them. Listing 11-3 shows how you can use this within
an interactive Python session.
#### Listing 11-3. Using pyximport

In [ ]:
import pyximport
pyximport.install()
import HelloWorld
print_msg()

This works fine for entire source files. If the section of code you wish to compile is
even shorter, you can have Cython compile it inline, directly in the middle of your Python
source code. Listing 11-4 gives one example of inlining compiled code.
#### Listing 11-4. Using Inlined Cython Code

In [ ]:
import cython
def my_adder(a, b):
ret = cython.inline("return a+b")

The compiled version of the inline code is cached in order to improve efficiency.
### 11-2. Using Static Types

You want to speed up access to objects by giving them a type.
<br>
You can install the Cython module, along with a supported C compiler, to define new
types that are accessed and worked with much faster than with Python objects.

To use static typing, Cython introduces a new keyword called cdef . When this is used,
you can get even more speedups than you achieved by compiling your Python code with
Cython. Listing 11-5 shows an example of an integration problem.
#### Listing 11-5. Pure Python Integration Problem

In [ ]:
def f(x):
    return x**2-42

def integrate_f(a, b, N):
s = 0
dx = (b-a)/N
for I in range(N):
    s += f(a+i*dx)
    return s*dx

Compiling this under Cython will provide a certain amount of speedup, but there
is still type checking that happens. This is especially costly on loops, where variables are
accessed many times. Listing 11-6 shows the same example, except using the cdef keyword.
#### Listing 11-6. Integration Problem Using Static Typing

In [ ]:
def f(double x):
    return x**2-42

def integrate_f(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b-a)/N
    for I in range(N):
        s += f(a+i*dx)
    return s*dx

This code removes all of those costly type checks and can be a great boon when
trying to optimize your code. In order to compile these files, you can use the Cython
command line utility to generate a C source file that can be compiled to a shared object
to be imported within Python. Assuming the above examples were saved in a file named
mycode.pyx , Listing 11-7 shows how to use GCC.
#### Listing 11-7. Compiling Cython Code Manually

In [ ]:
cython myfile.pyx
gcc -shared -o myfile.so myfile.c `python3-config --includes`

You can then import this newly compiled shared object from within Python.


### 11-3. Calling Python from C

You want to be able to call Python code from within a C program.
<br>
The standard library includes the header file called Python.h , which makes Python
callable from C.
<br>
There are two main functions that are available when you want to call Python code from
C: Py_Initialize() and Py_Finalize() . The first function starts the Python interpreter
and the second function shuts it down again. Between the two function calls, you can
run your Python code. Listing 11-8 shows an example where you can execute a string of
Python code.
#### Listing 11-8. Running Python Code from C

In [ ]:
#include "Python.h"
void run_pycode(const char* code) {
Py_Initialize();
PyRun_SimpleString(code);
Py_Finalize();
}

This works fine for shorter pieces of code, but if you have an entire script, you can
run it from your C program, as shown in Listing 11-9 .
#### Listing 11-9. Running a Python Script from C

In [ ]:
#include "Python.h"
Int main() {
Py_Initialize();
FILE* file = fopen("./my_script.py", "r");
PyRun_SimpleFile(file, "./my_script.py");
Py_Finalize();
}

### 11-4. Calling C from Python

You want to call external C code from a Python program.
<br>
The standard Python API includes code to help connect Python and C. The Cython
package makes this communication easier.

The keywords cdef extern from tell Cython a location from which to import C functions.
Listing 11-10 shows an example of a . pyx file.
#### Listing 11-10. Importing External C Code

In [ ]:
cdef extern from "hello_world.c":
void print_msg()
Listing 11-11 shows the related C source code file.
Listing 11-11. Imported C Code
static void print_msg() {
printf("Hello World");
}


This is a much simpler interface for importing C code than the standard API
included in Python.